In [46]:
import os

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import math
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import SpectralClustering

In [2]:
tech_env_data_df = pd.read_csv("./tech_env_dataset.csv")

In [3]:
tech_env_data_df = tech_env_data_df.rename(columns={"Unnamed: 0":"Index"})

In [4]:
for col in tech_env_data_df.columns:
    print(col)

Index
State
Area (Sq Mi)
Area (Sq Km)
Land (Sq Mi)
Land (Sq Km)
Land (%)
Water (Sq Mi)
Water (Sq Km)
Water (%)
population_2010
population_2011
population_2012
population_2013
population_2014
population_2015
population_2016
population_2017
population_2018
population_2019
state_funds_2012
fed_funds_2012
total_spent_2012
state_funds_2013
fed_funds_2013
total_spent_2013
state_funds_2014
fed_funds_2014
total_spent_2014
per_capita_2014
state_funds_2015
fed_funds_2015
total_spent_2015
per_capita_2015
state_funds_2016
fed_funds_2016
total_spent_2016
per_capita_2016
state_funds_2017
fed_funds_2017
total_spent_2017
per_capita_2017
desktop_use_2011
desktop_use_2013
desktop_use_2015
desktop_use_2017
desktop_use_2019
laptop_use_2011
laptop_use_2013
laptop_use_2015
laptop_use_2017
laptop_use_2019
tablet_use_2011
tablet_use_2013
tablet_use_2015
tablet_use_2017
tablet_use_2019
mobile_use_2011
mobile_use_2013
mobile_use_2015
mobile_use_2017
mobile_use_2019
smartTV_use_2011
smartTV_use_2013
smartTV_use_

In [5]:
def state_to_index(state):
    row = tech_env_data_df.loc[tech_env_data_df['State'] == state]
    return row.Index.values[0]

In [6]:
state_to_index('California')

4

In [7]:
law_data_df = pd.read_csv("./WebScrape/pew_bb_law_data.csv")

In [8]:
law_data_df = law_data_df.rename(columns={"Unnamed: 0":"Index"})

In [9]:
law = law_data_df[law_data_df["Index"] == 1]
law_data_df.head()

,Index,Header,State,Year,Title,Category,Topic,Summary,StateCode
0,0,Ala. Code 37-16-1 et seq.,Alabama,2019,Broadband Using Electric Easements Accessibili...,Category: Competition and regulation,Topic: Cooperatives,Allows electric utilities to also provide broa...,(a) To the extent not otherwise authorized by ...
1,1,Ala. Code 37-16-2,Alabama,2019,Broadband Using Electric Easements Accessibili...,Category: Other,Topic: Legislative intent,Declares that the state intends to encourage t...,(a) The Legislature finds and declares the fol...
2,2,Ala. Code 37-16-3,Alabama,2019,Broadband Using Electric Easements Accessibili...,Category: Definitions,Topic: Definition - Broadband (As defined by F...,"Defines ""advanced communications capabilities""...",(l) Advanced Communications Capabilities. The ...
3,3,Ala. Code 37-16-3,Alabama,2019,Broadband Using Electric Easements Accessibili...,Category: Definitions,Topic: Definition - Broadband (Other speed),Defines broadband as an internet connection th...,(5) Broadband Services. The provision of conne...
4,4,Ala. Code 37-16-7,Alabama,2019,Broadband Using Electric Easements Accessibili...,Category: Infrastructure access,Topic: Right of way (Easements),Specifies the terms under which a property own...,(a) If the owner of an interest in real proper...


In [10]:
laws = law_data_df[["Header", "Title", "State", "Year", "StateCode"]]
laws = laws.dropna()
index_names = laws[ laws['Year'] == 'no date' ].index
laws.drop(index_names, inplace = True)
laws['Year'].value_counts()



2018    160
2017    129
2019    106
2015     50
2013     30
2007     28
2011     22
2008     22
2016     21
2014     21
2005     19
2009     19
2012     18
2006     16
2004     13
2003     12
2001      9
1997      8
2010      7
2002      7
2000      7
1999      6
1996      2
1991      1
Name: Year, dtype: int64

In [11]:
descriptors = ["population", "state_funds", "fed_funds", "total_spent", "per_capita","Area (Sq Mi)", "Land (Sq Mi)", "Water (Sq Mi)", "Water (%)"]
metrics = ["desktop_use", "laptop_use", "tablet_use", "mobile_use", "smartTV_use", "wearable_use", "intUsers_above3", "intUsers_above15", "homeIntUsers", "workIntUsers", "schoolIntUsers", "cafeIntUsers", "altHomeIntUsers", "travelIntUsers", "publicIntUsers", "anyHomeIntUsers", "intAtHome", "noIntAtHome", "homeEverOnline", "noNeedInt", "noExpensiveInt", "noComputerInt", "useIntElsewhere", "noPrivSecInt", "unavailableInt", "mobileDataUsers", "wiredHighSpeedUsers", "satelliteUsers", "dialUpUsers", "intPrivateISP", "intPublicISP", "intIncluded", "intPublicFree", "emailUsers", "textIMUsers", "socialNetUsers", "publishUsers", "onlineConfUsers", "videoUsers", "teleworkUsers", "jobSearchUsers", "onlineClassUsers", "financeUsers", "eCommerceUsers", "sellingGoodsUsers", "offeringServiceUsers", "iotUsers"]

In [12]:
len(metrics)

47

In [13]:
cols_broken = {}
for name in metrics:
    years = []
    for text in tech_env_data_df.columns:
        if name in text:
            years += [text]
    cols_broken[name] = [years]
    

In [14]:
print(cols_broken['desktop_use'])

[['desktop_use_2011', 'desktop_use_2013', 'desktop_use_2015', 'desktop_use_2017', 'desktop_use_2019']]


In [15]:
obj = {}

for title, section in cols_broken.items():
    temp = []
    for name in section:
        temp += [tech_env_data_df[name]]
    obj[str(title)] = temp

In [16]:
obj.keys()

dict_keys(['desktop_use', 'laptop_use', 'tablet_use', 'mobile_use', 'smartTV_use', 'wearable_use', 'intUsers_above3', 'intUsers_above15', 'homeIntUsers', 'workIntUsers', 'schoolIntUsers', 'cafeIntUsers', 'altHomeIntUsers', 'travelIntUsers', 'publicIntUsers', 'anyHomeIntUsers', 'intAtHome', 'noIntAtHome', 'homeEverOnline', 'noNeedInt', 'noExpensiveInt', 'noComputerInt', 'useIntElsewhere', 'noPrivSecInt', 'unavailableInt', 'mobileDataUsers', 'wiredHighSpeedUsers', 'satelliteUsers', 'dialUpUsers', 'intPrivateISP', 'intPublicISP', 'intIncluded', 'intPublicFree', 'emailUsers', 'textIMUsers', 'socialNetUsers', 'publishUsers', 'onlineConfUsers', 'videoUsers', 'teleworkUsers', 'jobSearchUsers', 'onlineClassUsers', 'financeUsers', 'eCommerceUsers', 'sellingGoodsUsers', 'offeringServiceUsers', 'iotUsers'])

In [17]:
obj['onlineClassUsers']

[   onlineClassUsers_2001 onlineClassUsers_2003 onlineClassUsers_2015  \
 0                 42,988               115,490               509,455   
 1                 13,714                28,918               155,336   
 2                 93,293               178,035               803,095   
 3                 44,409                68,080               273,354   
 4                730,328             1,257,177             5,103,059   
 5                 79,182               164,088               937,840   
 6                 43,319                90,072               484,748   
 7                 13,195                24,625               104,890   
 8                320,595               632,837             2,779,458   
 9                159,753               219,402             1,133,575   
 10                16,027                36,449               186,950   
 11                21,310                46,795               239,976   
 12               181,217               348,009    

In [18]:
for key in obj.keys():
    cols = []
    for colname in obj[key][0].columns:
        numcol = int(colname[len(colname)-4:len(colname)])
        cols += [numcol]
    obj[key][0].columns = cols

In [19]:
obj['onlineClassUsers'][0].columns

Int64Index([2001, 2003, 2015, 2017, 2019], dtype='int64')

In [20]:
obj.keys()

dict_keys(['desktop_use', 'laptop_use', 'tablet_use', 'mobile_use', 'smartTV_use', 'wearable_use', 'intUsers_above3', 'intUsers_above15', 'homeIntUsers', 'workIntUsers', 'schoolIntUsers', 'cafeIntUsers', 'altHomeIntUsers', 'travelIntUsers', 'publicIntUsers', 'anyHomeIntUsers', 'intAtHome', 'noIntAtHome', 'homeEverOnline', 'noNeedInt', 'noExpensiveInt', 'noComputerInt', 'useIntElsewhere', 'noPrivSecInt', 'unavailableInt', 'mobileDataUsers', 'wiredHighSpeedUsers', 'satelliteUsers', 'dialUpUsers', 'intPrivateISP', 'intPublicISP', 'intIncluded', 'intPublicFree', 'emailUsers', 'textIMUsers', 'socialNetUsers', 'publishUsers', 'onlineConfUsers', 'videoUsers', 'teleworkUsers', 'jobSearchUsers', 'onlineClassUsers', 'financeUsers', 'eCommerceUsers', 'sellingGoodsUsers', 'offeringServiceUsers', 'iotUsers'])

In [21]:
print(obj['onlineClassUsers'][0][2017][0])
print(obj['onlineClassUsers'][0][2015][0])

485,862
509,455


In [22]:
def str2int(string):
    numeric_filter = filter(str.isdigit, string)
    numeric_string = "".join(numeric_filter)
    return int(numeric_string)

In [23]:
def label_maker(dct, mtc, st, yr):
    print(mtc, st, yr)
    #dct = dictionary, mtc = metric, st = state index, yr = year
    early = min(obj[mtc][0].columns)
    late = max(obj[mtc][0].columns)
    if (yr <= early) or (yr >= late):
        return -1
    else:
        yrs = obj[mtc][0].columns.values.tolist()
        if yr in yrs:
            nx_yr = yrs[next(x for x, val in enumerate(yrs) if val > yr)]
            delta = (str2int(obj[mtc][0][nx_yr][st]) - str2int(obj[mtc][0][yr][st])) / (nx_yr - yr)
            return delta
        else:
            nx_yr = yrs[next(x for x, val in enumerate(yrs) if val > yr)]
            temp = yrs.index(nx_yr)
            res = yrs[:temp]
            ls_yr = max(res)
            delta = (str2int(obj[mtc][0][nx_yr][st]) - str2int(obj[mtc][0][ls_yr][st])) / (nx_yr - ls_yr)
            return delta

In [24]:
x = label_maker(obj, 'onlineClassUsers', 5, 2017)
x

onlineClassUsers 5 2017


18923.0

In [25]:
for mtc in metrics:
    laws[mtc] = laws.apply(lambda row: label_maker(obj,mtc,state_to_index(row["State"]), int(row["Year"])), axis=1)

desktop_use 0 2019
desktop_use 0 2019
desktop_use 0 2019
desktop_use 0 2019
desktop_use 0 2019
desktop_use 0 2019
desktop_use 0 2018
desktop_use 0 2018
desktop_use 0 2018
desktop_use 0 2018
desktop_use 0 2017
desktop_use 0 2016
desktop_use 0 2016
desktop_use 0 2014
desktop_use 0 2012
desktop_use 0 2009
desktop_use 0 2000
desktop_use 0 2000
desktop_use 0 2000
desktop_use 1 2017
desktop_use 1 2017
desktop_use 2 2019
desktop_use 3 2019
desktop_use 3 2019
desktop_use 3 2019
desktop_use 3 2007
desktop_use 4 2019
desktop_use 4 2019
desktop_use 4 2019
desktop_use 4 2019
desktop_use 4 2019
desktop_use 4 2018
desktop_use 4 2018
desktop_use 4 2018
desktop_use 4 2017
desktop_use 4 2017
desktop_use 4 2017
desktop_use 4 2017
desktop_use 4 2017
desktop_use 4 2017
desktop_use 4 2017
desktop_use 4 2017
desktop_use 4 2016
desktop_use 4 2015
desktop_use 4 2015
desktop_use 4 2014
desktop_use 4 2014
desktop_use 4 2011
desktop_use 4 2009
desktop_use 4 2008
desktop_use 4 2007
desktop_use 4 2007
desktop_use 

desktop_use 39 2012
desktop_use 39 2012
desktop_use 39 2012
desktop_use 39 2012
desktop_use 39 2012
desktop_use 39 2006
desktop_use 39 2006
desktop_use 39 2002
desktop_use 39 2002
desktop_use 39 2002
desktop_use 40 2007
desktop_use 40 2007
desktop_use 40 1997
desktop_use 40 1997
desktop_use 41 2018
desktop_use 41 2018
desktop_use 41 2018
desktop_use 41 2017
desktop_use 41 2017
desktop_use 41 2017
desktop_use 41 2017
desktop_use 41 2017
desktop_use 41 2017
desktop_use 41 2017
desktop_use 41 2017
desktop_use 41 2017
desktop_use 41 2017
desktop_use 41 2017
desktop_use 41 2017
desktop_use 41 2017
desktop_use 41 2008
desktop_use 41 2008
desktop_use 41 2008
desktop_use 41 2008
desktop_use 41 2008
desktop_use 41 2008
desktop_use 41 2008
desktop_use 41 2008
desktop_use 41 2008
desktop_use 41 2008
desktop_use 41 2006
desktop_use 41 2006
desktop_use 42 2018
desktop_use 42 2017
desktop_use 42 2017
desktop_use 42 2017
desktop_use 42 2017
desktop_use 42 2017
desktop_use 42 2017
desktop_use 42 2017


laptop_use 14 2013
laptop_use 14 2013
laptop_use 14 2013
laptop_use 14 2004
laptop_use 15 2019
laptop_use 15 2018
laptop_use 15 2016
laptop_use 15 2013
laptop_use 15 2013
laptop_use 15 2013
laptop_use 15 2010
laptop_use 15 1996
laptop_use 16 2018
laptop_use 16 2018
laptop_use 16 2017
laptop_use 16 2015
laptop_use 16 2006
laptop_use 16 2006
laptop_use 16 2006
laptop_use 17 2019
laptop_use 17 2019
laptop_use 17 2017
laptop_use 17 2007
laptop_use 17 2004
laptop_use 17 2004
laptop_use 18 2019
laptop_use 18 2019
laptop_use 18 2019
laptop_use 18 2019
laptop_use 18 2019
laptop_use 18 2019
laptop_use 18 2019
laptop_use 18 2019
laptop_use 18 2019
laptop_use 18 2018
laptop_use 18 2018
laptop_use 18 2018
laptop_use 18 2017
laptop_use 18 2015
laptop_use 18 2015
laptop_use 18 2015
laptop_use 18 2015
laptop_use 18 2015
laptop_use 18 2015
laptop_use 18 2015
laptop_use 18 2015
laptop_use 18 2013
laptop_use 18 2013
laptop_use 18 2011
laptop_use 18 2011
laptop_use 18 2009
laptop_use 18 2005
laptop_use 1

laptop_use 48 2017
laptop_use 48 2015
laptop_use 48 2015
laptop_use 48 2015
laptop_use 48 2015
laptop_use 48 2011
laptop_use 48 2007
laptop_use 48 2005
laptop_use 48 2003
laptop_use 48 2003
laptop_use 49 2019
laptop_use 49 2019
laptop_use 49 2019
laptop_use 49 2019
laptop_use 49 2019
laptop_use 49 2019
laptop_use 49 2019
laptop_use 49 2019
laptop_use 49 2018
laptop_use 49 2018
laptop_use 49 2018
laptop_use 49 2018
laptop_use 49 2018
laptop_use 49 2018
laptop_use 49 2018
laptop_use 49 2018
laptop_use 49 2018
laptop_use 49 2018
laptop_use 49 2018
laptop_use 49 2018
laptop_use 49 2018
laptop_use 49 2018
laptop_use 49 2018
laptop_use 49 2018
laptop_use 10 2018
laptop_use 25 2018
laptop_use 29 2018
laptop_use 31 2018
laptop_use 36 2018
laptop_use 38 2018
tablet_use 0 2019
tablet_use 0 2019
tablet_use 0 2019
tablet_use 0 2019
tablet_use 0 2019
tablet_use 0 2019
tablet_use 0 2018
tablet_use 0 2018
tablet_use 0 2018
tablet_use 0 2018
tablet_use 0 2017
tablet_use 0 2016
tablet_use 0 2016
tablet

tablet_use 32 2018
tablet_use 32 2018
tablet_use 32 2018
tablet_use 32 2017
tablet_use 32 2017
tablet_use 32 2017
tablet_use 32 2017
tablet_use 32 2017
tablet_use 32 2017
tablet_use 32 2015
tablet_use 32 2015
tablet_use 32 2015
tablet_use 32 2015
tablet_use 32 2014
tablet_use 32 2013
tablet_use 32 2013
tablet_use 32 2013
tablet_use 32 2012
tablet_use 32 2011
tablet_use 32 2011
tablet_use 32 2011
tablet_use 32 2011
tablet_use 32 2011
tablet_use 32 2009
tablet_use 32 2007
tablet_use 32 2007
tablet_use 32 1999
tablet_use 33 1999
tablet_use 34 2015
tablet_use 34 2007
tablet_use 34 1996
tablet_use 35 2012
tablet_use 35 2002
tablet_use 35 1997
tablet_use 35 1997
tablet_use 36 2018
tablet_use 36 2018
tablet_use 36 2018
tablet_use 36 2018
tablet_use 36 2018
tablet_use 36 2018
tablet_use 36 2015
tablet_use 36 2015
tablet_use 36 2009
tablet_use 36 2003
tablet_use 36 2003
tablet_use 37 2017
tablet_use 37 2004
tablet_use 37 2004
tablet_use 37 2004
tablet_use 37 2004
tablet_use 37 2004
tablet_use 3

...

eCommerceUsers 4 2017
eCommerceUsers 4 2017
eCommerceUsers 4 2017
eCommerceUsers 4 2017
eCommerceUsers 4 2017
eCommerceUsers 4 2016
eCommerceUsers 4 2015
eCommerceUsers 4 2015
eCommerceUsers 4 2014
eCommerceUsers 4 2014
eCommerceUsers 4 2011
eCommerceUsers 4 2009
eCommerceUsers 4 2008
eCommerceUsers 4 2007
eCommerceUsers 4 2007
eCommerceUsers 4 2006
eCommerceUsers 4 2006
eCommerceUsers 4 2006
eCommerceUsers 4 2000
eCommerceUsers 5 2019
eCommerceUsers 5 2019
eCommerceUsers 5 2019
eCommerceUsers 5 2018
eCommerceUsers 5 2018
eCommerceUsers 5 2018
eCommerceUsers 5 2018
eCommerceUsers 5 2018
eCommerceUsers 5 2017
eCommerceUsers 5 2017
eCommerceUsers 5 2017
eCommerceUsers 5 2017
eCommerceUsers 5 2017
eCommerceUsers 5 2017
eCommerceUsers 5 2017
eCommerceUsers 5 2017
eCommerceUsers 5 2017
eCommerceUsers 5 2017
eCommerceUsers 5 2017
eCommerceUsers 5 2016
eCommerceUsers 5 2016
eCommerceUsers 5 2016
eCommerceUsers 5 2014
eCommerceUsers 5 2014
eCommerceUsers 5 2014
eCommerceUsers 5 2012
eCommerceU

eCommerceUsers 32 2018
eCommerceUsers 32 2018
eCommerceUsers 32 2018
eCommerceUsers 32 2018
eCommerceUsers 32 2018
eCommerceUsers 32 2018
eCommerceUsers 32 2017
eCommerceUsers 32 2017
eCommerceUsers 32 2017
eCommerceUsers 32 2017
eCommerceUsers 32 2017
eCommerceUsers 32 2017
eCommerceUsers 32 2015
eCommerceUsers 32 2015
eCommerceUsers 32 2015
eCommerceUsers 32 2015
eCommerceUsers 32 2014
eCommerceUsers 32 2013
eCommerceUsers 32 2013
eCommerceUsers 32 2013
eCommerceUsers 32 2012
eCommerceUsers 32 2011
eCommerceUsers 32 2011
eCommerceUsers 32 2011
eCommerceUsers 32 2011
eCommerceUsers 32 2011
eCommerceUsers 32 2009
eCommerceUsers 32 2007
eCommerceUsers 32 2007
eCommerceUsers 32 1999
eCommerceUsers 33 1999
eCommerceUsers 34 2015
eCommerceUsers 34 2007
eCommerceUsers 34 1996
eCommerceUsers 35 2012
eCommerceUsers 35 2002
eCommerceUsers 35 1997
eCommerceUsers 35 1997
eCommerceUsers 36 2018
eCommerceUsers 36 2018
eCommerceUsers 36 2018
eCommerceUsers 36 2018
eCommerceUsers 36 2018
eCommerceUs

sellingGoodsUsers 8 2005
sellingGoodsUsers 8 2000
sellingGoodsUsers 9 2019
sellingGoodsUsers 9 2019
sellingGoodsUsers 9 2019
sellingGoodsUsers 9 2019
sellingGoodsUsers 9 2019
sellingGoodsUsers 9 2018
sellingGoodsUsers 9 2018
sellingGoodsUsers 9 2018
sellingGoodsUsers 9 2018
sellingGoodsUsers 9 2018
sellingGoodsUsers 9 2018
sellingGoodsUsers 9 2018
sellingGoodsUsers 9 2018
sellingGoodsUsers 9 2018
sellingGoodsUsers 9 2018
sellingGoodsUsers 9 2018
sellingGoodsUsers 9 2018
sellingGoodsUsers 9 2018
sellingGoodsUsers 9 2018
sellingGoodsUsers 9 2018
sellingGoodsUsers 9 2018
sellingGoodsUsers 9 2018
sellingGoodsUsers 9 2018
sellingGoodsUsers 9 2018
sellingGoodsUsers 9 2018
sellingGoodsUsers 9 2018
sellingGoodsUsers 9 2018
sellingGoodsUsers 9 2018
sellingGoodsUsers 9 2018
sellingGoodsUsers 9 2018
sellingGoodsUsers 9 2014
sellingGoodsUsers 9 2010
sellingGoodsUsers 9 2006
sellingGoodsUsers 10 2019
sellingGoodsUsers 10 2016
sellingGoodsUsers 10 2014
sellingGoodsUsers 10 2014
sellingGoodsUsers 10 

sellingGoodsUsers 32 2018
sellingGoodsUsers 32 2018
sellingGoodsUsers 32 2018
sellingGoodsUsers 32 2018
sellingGoodsUsers 32 2018
sellingGoodsUsers 32 2018
sellingGoodsUsers 32 2017
sellingGoodsUsers 32 2017
sellingGoodsUsers 32 2017
sellingGoodsUsers 32 2017
sellingGoodsUsers 32 2017
sellingGoodsUsers 32 2017
sellingGoodsUsers 32 2015
sellingGoodsUsers 32 2015
sellingGoodsUsers 32 2015
sellingGoodsUsers 32 2015
sellingGoodsUsers 32 2014
sellingGoodsUsers 32 2013
sellingGoodsUsers 32 2013
sellingGoodsUsers 32 2013
sellingGoodsUsers 32 2012
sellingGoodsUsers 32 2011
sellingGoodsUsers 32 2011
sellingGoodsUsers 32 2011
sellingGoodsUsers 32 2011
sellingGoodsUsers 32 2011
sellingGoodsUsers 32 2009
sellingGoodsUsers 32 2007
sellingGoodsUsers 32 2007
sellingGoodsUsers 32 1999
sellingGoodsUsers 33 1999
sellingGoodsUsers 34 2015
sellingGoodsUsers 34 2007
sellingGoodsUsers 34 1996
sellingGoodsUsers 35 2012
sellingGoodsUsers 35 2002
sellingGoodsUsers 35 1997
sellingGoodsUsers 35 1997
sellingGoods

ValueError: min() arg is an empty sequence

In [26]:
laws

,Header,Title,State,Year,StateCode,desktop_use,laptop_use,tablet_use,mobile_use,smartTV_use,...,socialNetUsers,publishUsers,onlineConfUsers,videoUsers,teleworkUsers,jobSearchUsers,onlineClassUsers,financeUsers,eCommerceUsers,sellingGoodsUsers
0,Ala. Code 37-16-1 et seq.,Broadband Using Electric Easements Accessibili...,Alabama,2019,(a) To the extent not otherwise authorized by ...,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
1,Ala. Code 37-16-2,Broadband Using Electric Easements Accessibili...,Alabama,2019,(a) The Legislature finds and declares the fol...,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2,Ala. Code 37-16-3,Broadband Using Electric Easements Accessibili...,Alabama,2019,(l) Advanced Communications Capabilities. The ...,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
3,Ala. Code 37-16-3,Broadband Using Electric Easements Accessibili...,Alabama,2019,(5) Broadband Services. The provision of conne...,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
4,Ala. Code 37-16-7,Broadband Using Electric Easements Accessibili...,Alabama,2019,(a) If the owner of an interest in real proper...,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
846,Executive Order No. 3-2018,Executive order providing for Internet neutral...,Montana,2018,"WHEREAS, the free and open exchange of informa...",-10146.0,30650.0,13521.5,32914.0,51884.5,...,14925.0,-13313.5,32062.0,38898.0,17051.5,14777.0,3339.0,17342.0,12187.5,-3272.0
847,Executive Order No. 9,Requires that all future contracts for Interne...,New Jersey,2018,"WHEREAS, a free and open Internet is essential...",-108421.5,-6186.0,-69868.5,281387.5,145523.5,...,43384.5,129039.0,156843.5,137053.0,226790.5,73846.5,190194.5,270223.5,326193.5,105806.0
848,Executive Order No. 175,Ensuring Net Neutrality Protections for New Yo...,New York,2018,"WHEREAS, the internet is an essential service ...",-497152.0,53077.0,-36395.5,286738.5,584155.5,...,80479.5,7105.0,407389.0,303792.0,61245.0,-206324.0,-44200.0,187836.0,184027.0,42437.0
849,Or. Rev. Stat. 276A.418,Public contracts for broadband Internet access...,Oregon,2018,"(2) For the purposes of this section, a public...",-102803.0,-40436.5,-38204.5,68032.5,116553.5,...,75860.5,2296.5,57330.0,78378.0,-27353.5,12585.0,-20832.5,46377.5,67942.0,-79734.5


In [27]:
descriptors
dynamic_descriptors = ['population', 'state_funds', 'fed_funds', 'total_spent', 'per_capita']
static_descriptors = ['Area (Sq Mi)', 'Land (Sq Mi)', 'Water (Sq Mi)', 'Water (%)']

In [28]:
cols_broken = {}
for name in descriptors:
    years = []
    for text in tech_env_data_df.columns:
        if name in text:
            years += [text]
    cols_broken[name] = [years]

In [29]:
env = {}

for title, section in cols_broken.items():
    temp = []
    for name in section:
        temp += [tech_env_data_df[name]]
    env[str(title)] = temp

In [30]:
env

{'population': [   population_2010 population_2011 population_2012 population_2013  \
  0        4,785,437       4,799,069       4,815,588       4,830,081   
  1          713,910         722,128         730,443         737,068   
  2        6,407,172       6,472,643       6,554,978       6,632,764   
  3        2,921,964       2,940,667       2,952,164       2,959,400   
  4       37,319,502      37,638,369      37,948,800      38,260,787   
  5        5,047,349       5,121,108       5,192,647       5,269,035   
  6        3,579,114       3,588,283       3,594,547       3,594,841   
  7          899,593         907,381         915,179         923,576   
  8       18,845,537      19,053,237      19,297,822      19,545,621   
  9        9,711,881       9,802,431       9,901,430       9,972,479   
  10       1,363,963       1,379,329       1,394,804       1,408,243   
  11       1,570,746       1,583,910       1,595,324       1,611,206   
  12      12,840,503      12,867,454      12,882,5

In [31]:
for key in env.keys():
    cols = []
    for colname in env[key][0].columns:
        numcol = int(colname[len(colname)-4:len(colname)])
        cols += [numcol]
    env[key][0].columns = cols

ValueError: invalid literal for int() with base 10: ' Mi)'

In [32]:
env.keys()

dict_keys(['population', 'state_funds', 'fed_funds', 'total_spent', 'per_capita', 'Area (Sq Mi)', 'Land (Sq Mi)', 'Water (Sq Mi)', 'Water (%)'])

In [33]:
def attach_env(dct, mtc, st, yr):
    print(mtc, st, yr)
    #dct = dictionary, mtc = metric, st = state index, yr = year
    if mtc in static_descriptors:
        return str2int(dct[mtc][0].iloc(st)[0].values[0])
    else:
        yrs = dct[mtc][0].columns.values.tolist()
        if yr in yrs:
            return str2int(dct[mtc][0][yr][0])
        else:
            diff = [abs(x - yr) for x in yrs]
            lowest = diff.index(min(diff))
            print(lowest)
            return str2int(dct[mtc][0][yrs[lowest]][0])

In [34]:
env['population'][0]

,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,"4,785,437","4,799,069","4,815,588","4,830,081","4,841,799","4,852,347","4,863,525","4,874,486","4,887,681","4,903,185"
1,"713,910","722,128","730,443","737,068","736,283","737,498","741,456","739,700","735,139","731,545"
2,"6,407,172","6,472,643","6,554,978","6,632,764","6,730,413","6,829,676","6,941,072","7,044,008","7,158,024","7,278,717"
3,"2,921,964","2,940,667","2,952,164","2,959,400","2,967,392","2,978,048","2,989,918","3,001,345","3,009,733","3,017,804"
4,"37,319,502","37,638,369","37,948,800","38,260,787","38,596,972","38,918,045","39,167,117","39,358,497","39,461,588","39,512,223"
5,"5,047,349","5,121,108","5,192,647","5,269,035","5,350,101","5,450,623","5,539,215","5,611,885","5,691,287","5,758,736"
6,"3,579,114","3,588,283","3,594,547","3,594,841","3,594,524","3,587,122","3,578,141","3,573,297","3,571,520","3,565,287"
7,"899,593","907,381","915,179","923,576","932,487","941,252","948,921","956,823","965,479","973,764"
8,"18,845,537","19,053,237","19,297,822","19,545,621","19,845,911","20,209,042","20,613,477","20,963,613","21,244,317","21,477,737"
9,"9,711,881","9,802,431","9,901,430","9,972,479","10,067,278","10,178,447","10,301,890","10,410,330","10,511,131","10,617,423"


In [35]:
attach_env(env, 'population', 0, 2015)

population 0 2015


4852347

In [36]:
for mtc in descriptors:
    laws[mtc] = laws.apply(lambda row: attach_env(env,mtc,state_to_index(row["State"]), int(row["Year"])), axis=1)

population 0 2019
population 0 2019
population 0 2019
population 0 2019
population 0 2019
population 0 2019
population 0 2018
population 0 2018
population 0 2018
population 0 2018
population 0 2017
population 0 2016
population 0 2016
population 0 2014
population 0 2012
population 0 2009
0
population 0 2000
0
population 0 2000
0
population 0 2000
0
population 1 2017
population 1 2017
population 2 2019
population 3 2019
population 3 2019
population 3 2019
population 3 2007
0
population 4 2019
population 4 2019
population 4 2019
population 4 2019
population 4 2019
population 4 2018
population 4 2018
population 4 2018
population 4 2017
population 4 2017
population 4 2017
population 4 2017
population 4 2017
population 4 2017
population 4 2017
population 4 2017
population 4 2016
population 4 2015
population 4 2015
population 4 2014
population 4 2014
population 4 2011
population 4 2009
0
population 4 2008
0
population 4 2007
0
population 4 2007
0
population 4 2006
0
population 4 2006
0
popula

population 41 2006
0
population 42 2018
population 42 2017
population 42 2017
population 42 2017
population 42 2017
population 42 2017
population 42 2017
population 42 2017
population 42 2013
population 42 2005
0
population 42 2005
0
population 42 2003
0
population 42 1999
0
population 42 1997
0
population 42 1997
0
population 43 2018
population 43 2018
population 43 2018
population 43 2018
population 43 2018
population 43 2018
population 43 2018
population 43 2018
population 43 2018
population 43 2018
population 43 2017
population 43 2017
population 43 2017
population 43 2001
0
population 43 2001
0
population 43 2001
0
population 44 2019
population 44 2019
population 44 2019
population 44 2019
population 44 2019
population 44 2019
population 44 2019
population 44 2019
population 44 2018
population 44 2017
population 44 2017
population 44 2017
population 44 2017
population 44 2017
population 44 2017
population 44 2017
population 44 2015
population 44 2015
population 44 2015
population 

state_funds 17 2017
state_funds 17 2007
0
state_funds 17 2004
0
state_funds 17 2004
0
state_funds 18 2019
5
state_funds 18 2019
5
state_funds 18 2019
5
state_funds 18 2019
5
state_funds 18 2019
5
state_funds 18 2019
5
state_funds 18 2019
5
state_funds 18 2019
5
state_funds 18 2019
5
state_funds 18 2018
5
state_funds 18 2018
5
state_funds 18 2018
5
state_funds 18 2017
state_funds 18 2015
state_funds 18 2015
state_funds 18 2015
state_funds 18 2015
state_funds 18 2015
state_funds 18 2015
state_funds 18 2015
state_funds 18 2015
state_funds 18 2013
state_funds 18 2013
state_funds 18 2011
0
state_funds 18 2011
0
state_funds 18 2009
0
state_funds 18 2005
0
state_funds 18 2005
0
state_funds 18 2005
0
state_funds 18 2005
0
state_funds 19 2019
5
state_funds 19 2019
5
state_funds 19 2019
5
state_funds 19 2019
5
state_funds 19 2017
state_funds 19 2017
state_funds 20 2018
5
state_funds 20 2017
state_funds 20 2017
state_funds 20 2008
0
state_funds 20 2008
0
state_funds 20 2008
0
state_funds 20 2008


state_funds 46 2013
state_funds 46 2011
0
state_funds 46 2011
0
state_funds 46 2011
0
state_funds 46 2011
0
state_funds 46 2011
0
state_funds 46 2011
0
state_funds 46 2008
0
state_funds 47 2019
5
state_funds 47 2019
5
state_funds 47 2019
5
state_funds 47 2019
5
state_funds 47 2019
5
state_funds 47 2019
5
state_funds 47 2019
5
state_funds 47 2018
5
state_funds 47 2018
5
state_funds 47 2018
5
state_funds 47 2017
state_funds 47 2017
state_funds 47 2017
state_funds 47 2017
state_funds 47 2017
state_funds 47 2017
state_funds 47 2017
state_funds 47 2017
state_funds 47 2017
state_funds 47 2017
state_funds 47 2017
state_funds 47 2017
state_funds 47 2017
state_funds 47 2017
state_funds 47 2017
state_funds 47 2014
state_funds 47 2013
state_funds 48 2019
5
state_funds 48 2019
5
state_funds 48 2017
state_funds 48 2017
state_funds 48 2017
state_funds 48 2017
state_funds 48 2015
state_funds 48 2015
state_funds 48 2015
state_funds 48 2015
state_funds 48 2011
0
state_funds 48 2007
0
state_funds 48 200

fed_funds 26 2019
5
fed_funds 26 2018
5
fed_funds 26 2018
5
fed_funds 26 2018
5
fed_funds 26 2018
5
fed_funds 26 2018
5
fed_funds 26 2005
0
fed_funds 26 2005
0
fed_funds 26 2005
0
fed_funds 26 2005
0
fed_funds 26 2001
0
fed_funds 26 2001
0
fed_funds 26 2001
0
fed_funds 27 2017
fed_funds 27 2017
fed_funds 27 2017
fed_funds 27 2017
fed_funds 27 2017
fed_funds 27 2017
fed_funds 27 2003
0
fed_funds 27 2003
0
fed_funds 27 1997
0
fed_funds 27 1997
0
fed_funds 28 2018
5
fed_funds 28 2018
5
fed_funds 28 2018
5
fed_funds 28 2018
5
fed_funds 28 2006
0
fed_funds 28 2006
0
fed_funds 28 2006
0
fed_funds 28 2006
0
fed_funds 29 2019
5
fed_funds 29 2007
0
fed_funds 29 2007
0
fed_funds 29 2007
0
fed_funds 29 2007
0
fed_funds 29 2007
0
fed_funds 29 2007
0
fed_funds 29 2007
0
fed_funds 29 2007
0
fed_funds 29 2007
0
fed_funds 29 2007
0
fed_funds 30 2018
5
fed_funds 30 2018
5
fed_funds 30 2017
fed_funds 30 2017
fed_funds 30 2017
fed_funds 30 2017
fed_funds 30 2017
fed_funds 30 2017
fed_funds 30 2017
fed_fu

total_spent 5 2019
5
total_spent 5 2019
5
total_spent 5 2019
5
total_spent 5 2018
5
total_spent 5 2018
5
total_spent 5 2018
5
total_spent 5 2018
5
total_spent 5 2018
5
total_spent 5 2017
total_spent 5 2017
total_spent 5 2017
total_spent 5 2017
total_spent 5 2017
total_spent 5 2017
total_spent 5 2017
total_spent 5 2017
total_spent 5 2017
total_spent 5 2017
total_spent 5 2017
total_spent 5 2016
total_spent 5 2016
total_spent 5 2016
total_spent 5 2014
total_spent 5 2014
total_spent 5 2014
total_spent 5 2012
total_spent 5 2008
0
total_spent 5 2005
0
total_spent 5 2005
0
total_spent 5 2005
0
total_spent 5 2005
0
total_spent 5 2005
0
total_spent 5 2000
0
total_spent 6 2019
5
total_spent 6 2016
total_spent 6 2015
total_spent 7 2014
total_spent 7 2013
total_spent 7 2013
total_spent 8 2019
5
total_spent 8 2017
total_spent 8 2017
total_spent 8 2012
total_spent 8 2012
total_spent 8 2011
0
total_spent 8 2009
0
total_spent 8 2005
0
total_spent 8 2000
0
total_spent 9 2019
5
total_spent 9 2019
5
tota

total_spent 42 2017
total_spent 42 2017
total_spent 42 2013
total_spent 42 2005
0
total_spent 42 2005
0
total_spent 42 2003
0
total_spent 42 1999
0
total_spent 42 1997
0
total_spent 42 1997
0
total_spent 43 2018
5
total_spent 43 2018
5
total_spent 43 2018
5
total_spent 43 2018
5
total_spent 43 2018
5
total_spent 43 2018
5
total_spent 43 2018
5
total_spent 43 2018
5
total_spent 43 2018
5
total_spent 43 2018
5
total_spent 43 2017
total_spent 43 2017
total_spent 43 2017
total_spent 43 2001
0
total_spent 43 2001
0
total_spent 43 2001
0
total_spent 44 2019
5
total_spent 44 2019
5
total_spent 44 2019
5
total_spent 44 2019
5
total_spent 44 2019
5
total_spent 44 2019
5
total_spent 44 2019
5
total_spent 44 2019
5
total_spent 44 2018
5
total_spent 44 2017
total_spent 44 2017
total_spent 44 2017
total_spent 44 2017
total_spent 44 2017
total_spent 44 2017
total_spent 44 2017
total_spent 44 2015
total_spent 44 2015
total_spent 44 2015
total_spent 44 2015
total_spent 44 2015
total_spent 44 2015
tota

per_capita 18 2018
3
per_capita 18 2017
per_capita 18 2015
per_capita 18 2015
per_capita 18 2015
per_capita 18 2015
per_capita 18 2015
per_capita 18 2015
per_capita 18 2015
per_capita 18 2015
per_capita 18 2013
0
per_capita 18 2013
0
per_capita 18 2011
0
per_capita 18 2011
0
per_capita 18 2009
0
per_capita 18 2005
0
per_capita 18 2005
0
per_capita 18 2005
0
per_capita 18 2005
0
per_capita 19 2019
3
per_capita 19 2019
3
per_capita 19 2019
3
per_capita 19 2019
3
per_capita 19 2017
per_capita 19 2017
per_capita 20 2018
3
per_capita 20 2017
per_capita 20 2017
per_capita 20 2008
0
per_capita 20 2008
0
per_capita 20 2008
0
per_capita 20 2008
0
per_capita 20 2008
0
per_capita 20 2008
0
per_capita 20 2008
0
per_capita 20 2008
0
per_capita 21 2018
3
per_capita 21 2018
3
per_capita 21 2018
3
per_capita 21 2018
3
per_capita 21 2018
3
per_capita 21 2018
3
per_capita 21 2017
per_capita 21 2017
per_capita 21 2014
per_capita 21 2014
per_capita 21 2014
per_capita 21 2012
0
per_capita 21 2011
0
per_cap

3
per_capita 49 2018
3
per_capita 49 2018
3
per_capita 49 2018
3
per_capita 49 2018
3
per_capita 49 2018
3
per_capita 49 2018
3
per_capita 49 2018
3
per_capita 49 2018
3
per_capita 49 2018
3
per_capita 49 2018
3
per_capita 49 2018
3
per_capita 49 2018
3
per_capita 49 2018
3
per_capita 49 2018
3
per_capita 49 2018
3
per_capita 10 2018
3
per_capita 25 2018
3
per_capita 29 2018
3
per_capita 31 2018
3
per_capita 36 2018
3
per_capita 38 2018
3
Area (Sq Mi) 0 2019
Area (Sq Mi) 0 2019
Area (Sq Mi) 0 2019
Area (Sq Mi) 0 2019
Area (Sq Mi) 0 2019
Area (Sq Mi) 0 2019
Area (Sq Mi) 0 2018
Area (Sq Mi) 0 2018
Area (Sq Mi) 0 2018
Area (Sq Mi) 0 2018
Area (Sq Mi) 0 2017
Area (Sq Mi) 0 2016
Area (Sq Mi) 0 2016
Area (Sq Mi) 0 2014
Area (Sq Mi) 0 2012
Area (Sq Mi) 0 2009
Area (Sq Mi) 0 2000
Area (Sq Mi) 0 2000
Area (Sq Mi) 0 2000
Area (Sq Mi) 1 2017
Area (Sq Mi) 1 2017
Area (Sq Mi) 2 2019


ValueError: No axis named 2 for object type <class 'pandas.core.frame.DataFrame'>

In [37]:
laws

,Header,Title,State,Year,StateCode,desktop_use,laptop_use,tablet_use,mobile_use,smartTV_use,...,jobSearchUsers,onlineClassUsers,financeUsers,eCommerceUsers,sellingGoodsUsers,population,state_funds,fed_funds,total_spent,per_capita
0,Ala. Code 37-16-1 et seq.,Broadband Using Electric Easements Accessibili...,Alabama,2019,(a) To the extent not otherwise authorized by ...,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,4903185,15524,9911,25435,5218
1,Ala. Code 37-16-2,Broadband Using Electric Easements Accessibili...,Alabama,2019,(a) The Legislature finds and declares the fol...,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,4903185,15524,9911,25435,5218
2,Ala. Code 37-16-3,Broadband Using Electric Easements Accessibili...,Alabama,2019,(l) Advanced Communications Capabilities. The ...,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,4903185,15524,9911,25435,5218
3,Ala. Code 37-16-3,Broadband Using Electric Easements Accessibili...,Alabama,2019,(5) Broadband Services. The provision of conne...,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,4903185,15524,9911,25435,5218
4,Ala. Code 37-16-7,Broadband Using Electric Easements Accessibili...,Alabama,2019,(a) If the owner of an interest in real proper...,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,-1.0,4903185,15524,9911,25435,5218
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
846,Executive Order No. 3-2018,Executive order providing for Internet neutral...,Montana,2018,"WHEREAS, the free and open exchange of informa...",-10146.0,30650.0,13521.5,32914.0,51884.5,...,14777.0,3339.0,17342.0,12187.5,-3272.0,4887681,15524,9911,25435,5218
847,Executive Order No. 9,Requires that all future contracts for Interne...,New Jersey,2018,"WHEREAS, a free and open Internet is essential...",-108421.5,-6186.0,-69868.5,281387.5,145523.5,...,73846.5,190194.5,270223.5,326193.5,105806.0,4887681,15524,9911,25435,5218
848,Executive Order No. 175,Ensuring Net Neutrality Protections for New Yo...,New York,2018,"WHEREAS, the internet is an essential service ...",-497152.0,53077.0,-36395.5,286738.5,584155.5,...,-206324.0,-44200.0,187836.0,184027.0,42437.0,4887681,15524,9911,25435,5218
849,Or. Rev. Stat. 276A.418,Public contracts for broadband Internet access...,Oregon,2018,"(2) For the purposes of this section, a public...",-102803.0,-40436.5,-38204.5,68032.5,116553.5,...,12585.0,-20832.5,46377.5,67942.0,-79734.5,4887681,15524,9911,25435,5218


# Clustering usign cosine similarity matrix

In [40]:
stateCodes = laws.StateCode.dropna()
stateCodes

0      (a) To the extent not otherwise authorized by ...
1      (a) The Legislature finds and declares the fol...
2      (l) Advanced Communications Capabilities. The ...
3      (5) Broadband Services. The provision of conne...
4      (a) If the owner of an interest in real proper...
                             ...                        
846    WHEREAS, the free and open exchange of informa...
847    WHEREAS, a free and open Internet is essential...
848    WHEREAS, the internet is an essential service ...
849    (2) For the purposes of this section, a public...
850    WHEREAS, the free and open internet is an esse...
Name: StateCode, Length: 733, dtype: object

In [79]:
vect = TfidfVectorizer(min_df=1, stop_words="english", encoding='latin-1')
tfidf = vect.fit_transform(stateCodes)
pairwise_similarity = tfidf * tfidf.T

In [80]:
type(pairwise_similarity)

scipy.sparse.csr.csr_matrix

In [155]:
clustered = SpectralClustering(n_clusters=10, affinity='nearest_neighbors', n_neighbors=100).fit_predict(pairwise_similarity)
y = np.bincount(clustered)
y

/Users/dominickperini/opt/anaconda3/lib/python3.8/site-packages/sklearn/cluster/_spectral.py:484: UserWarning: The spectral clustering API has changed. ``fit``now constructs an affinity matrix from data. To use a custom affinity matrix, set ``affinity=precomputed``.
  warnings.warn("The spectral clustering API has changed. ``fit``"


array([ 83, 109,  57,  68,  44, 167,  40,  68,  37,  60])

In [153]:
from sklearn.cluster import DBSCAN


In [67]:
pairwise_similarity_a = pairwise_similarity.toarray()

In [70]:
# Create a function that converts from similarity to distance
invert = lambda i: 1-i

# Create a vectorized function
v_invert = np.vectorize(invert)

In [71]:
pairwise_distance = v_invert(pairwise_similarity_a)

In [73]:
DBSCAN(min_samples=1).fit_predict(pairwise_distance)

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  11,
        12,  13,  14,  15,  15,  16,  17,  18,  19,  20,  21,  22,  23,
        24,  25,  26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,
        37,  38,  39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,
        50,  51,  52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,
        63,  64,  65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,
        76,  77,  78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,
        89,  90,  91,  90,  92,  93,  94,  95,  96,  97,  98,  99, 100,
       101, 102, 103, 104, 105, 106, 107, 108, 109, 109, 110, 110, 111,
       112, 113, 114, 114, 115, 116, 117, 117, 118, 118, 119, 120, 121,
       122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134,
       135, 136, 137, 138, 139, 140, 141, 141, 142, 143, 144, 145, 146,
       147, 148, 149, 150, 149, 151, 152, 153, 154, 155, 156, 157, 158,
       159, 160, 161, 161, 162, 163, 164, 165, 166, 167, 168, 16

## Append one-hot encoded groups to laws

In [170]:
for i in range(len(laws)):
    x = 1

clustered

array([1, 7, 1, 4, 5, 3, 2, 4, 8, 6, 5, 5, 5, 1, 9, 3, 1, 1, 1, 6, 5, 5,
       7, 3, 5, 1, 5, 6, 0, 0, 5, 9, 9, 1, 6, 5, 7, 4, 5, 5, 5, 8, 0, 0,
       2, 9, 0, 1, 5, 1, 1, 2, 3, 4, 7, 9, 1, 9, 9, 2, 4, 9, 2, 8, 3, 3,
       3, 1, 3, 3, 3, 3, 1, 0, 1, 5, 2, 6, 0, 2, 1, 6, 5, 1, 4, 1, 1, 1,
       1, 3, 5, 7, 8, 8, 5, 5, 3, 3, 7, 7, 1, 1, 9, 7, 9, 5, 5, 5, 3, 1,
       4, 2, 7, 7, 0, 0, 0, 0, 0, 0, 0, 5, 9, 0, 0, 0, 0, 5, 0, 0, 0, 0,
       0, 7, 3, 1, 9, 6, 0, 5, 7, 7, 0, 0, 9, 5, 5, 2, 5, 0, 0, 1, 5, 6,
       6, 5, 5, 7, 8, 5, 8, 5, 8, 1, 0, 0, 0, 8, 7, 9, 2, 6, 6, 7, 3, 5,
       5, 1, 1, 8, 1, 1, 1, 4, 1, 5, 5, 5, 7, 4, 4, 4, 2, 2, 4, 5, 1, 5,
       1, 5, 0, 3, 0, 0, 0, 2, 9, 2, 2, 5, 2, 2, 5, 0, 7, 7, 5, 3, 9, 5,
       2, 1, 3, 6, 3, 4, 1, 1, 5, 1, 0, 0, 2, 0, 8, 7, 7, 6, 6, 1, 5, 1,
       4, 6, 8, 3, 2, 5, 5, 9, 5, 0, 0, 1, 5, 8, 1, 2, 7, 2, 8, 7, 5, 2,
       5, 5, 1, 8, 5, 5, 5, 1, 1, 7, 5, 1, 1, 7, 5, 2, 0, 5, 9, 7, 6, 0,
       5, 7, 5, 8, 1, 0, 1, 5, 5, 5, 4, 2, 1, 4, 9,

In [172]:
laws2 = laws

In [173]:
laws2['cluster'] = clustered.tolist()

In [174]:
laws2

,Header,Title,State,Year,StateCode,desktop_use,laptop_use,tablet_use,mobile_use,smartTV_use,...,onlineClassUsers,financeUsers,eCommerceUsers,sellingGoodsUsers,population,state_funds,fed_funds,total_spent,per_capita,cluster
0,Ala. Code 37-16-1 et seq.,Broadband Using Electric Easements Accessibili...,Alabama,2019,(a) To the extent not otherwise authorized by ...,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,4903185,15524,9911,25435,5218,1
1,Ala. Code 37-16-2,Broadband Using Electric Easements Accessibili...,Alabama,2019,(a) The Legislature finds and declares the fol...,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,4903185,15524,9911,25435,5218,7
2,Ala. Code 37-16-3,Broadband Using Electric Easements Accessibili...,Alabama,2019,(l) Advanced Communications Capabilities. The ...,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,4903185,15524,9911,25435,5218,1
3,Ala. Code 37-16-3,Broadband Using Electric Easements Accessibili...,Alabama,2019,(5) Broadband Services. The provision of conne...,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,4903185,15524,9911,25435,5218,4
4,Ala. Code 37-16-7,Broadband Using Electric Easements Accessibili...,Alabama,2019,(a) If the owner of an interest in real proper...,-1.0,-1.0,-1.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,-1.0,4903185,15524,9911,25435,5218,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
846,Executive Order No. 3-2018,Executive order providing for Internet neutral...,Montana,2018,"WHEREAS, the free and open exchange of informa...",-10146.0,30650.0,13521.5,32914.0,51884.5,...,3339.0,17342.0,12187.5,-3272.0,4887681,15524,9911,25435,5218,7
847,Executive Order No. 9,Requires that all future contracts for Interne...,New Jersey,2018,"WHEREAS, a free and open Internet is essential...",-108421.5,-6186.0,-69868.5,281387.5,145523.5,...,190194.5,270223.5,326193.5,105806.0,4887681,15524,9911,25435,5218,9
848,Executive Order No. 175,Ensuring Net Neutrality Protections for New Yo...,New York,2018,"WHEREAS, the internet is an essential service ...",-497152.0,53077.0,-36395.5,286738.5,584155.5,...,-44200.0,187836.0,184027.0,42437.0,4887681,15524,9911,25435,5218,9
849,Or. Rev. Stat. 276A.418,Public contracts for broadband Internet access...,Oregon,2018,"(2) For the purposes of this section, a public...",-102803.0,-40436.5,-38204.5,68032.5,116553.5,...,-20832.5,46377.5,67942.0,-79734.5,4887681,15524,9911,25435,5218,9


In [175]:
ohe = pd.get_dummies(laws2.cluster, prefix='Cluster_')

In [176]:
ohe

,Cluster__0,Cluster__1,Cluster__2,Cluster__3,Cluster__4,Cluster__5,Cluster__6,Cluster__7,Cluster__8,Cluster__9
0,0,1,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,1,0,0
2,0,1,0,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0,0,0,0
4,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
846,0,0,0,0,0,0,0,1,0,0
847,0,0,0,0,0,0,0,0,0,1
848,0,0,0,0,0,0,0,0,0,1
849,0,0,0,0,0,0,0,0,0,1


In [177]:
df = pd.concat([ohe, laws], axis=1)

In [178]:
df

,Cluster__0,Cluster__1,Cluster__2,Cluster__3,Cluster__4,Cluster__5,Cluster__6,Cluster__7,Cluster__8,Cluster__9,...,onlineClassUsers,financeUsers,eCommerceUsers,sellingGoodsUsers,population,state_funds,fed_funds,total_spent,per_capita,cluster
0,0,1,0,0,0,0,0,0,0,0,...,-1.0,-1.0,-1.0,-1.0,4903185,15524,9911,25435,5218,1
1,0,0,0,0,0,0,0,1,0,0,...,-1.0,-1.0,-1.0,-1.0,4903185,15524,9911,25435,5218,7
2,0,1,0,0,0,0,0,0,0,0,...,-1.0,-1.0,-1.0,-1.0,4903185,15524,9911,25435,5218,1
3,0,0,0,0,1,0,0,0,0,0,...,-1.0,-1.0,-1.0,-1.0,4903185,15524,9911,25435,5218,4
4,0,0,0,0,0,1,0,0,0,0,...,-1.0,-1.0,-1.0,-1.0,4903185,15524,9911,25435,5218,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
846,0,0,0,0,0,0,0,1,0,0,...,3339.0,17342.0,12187.5,-3272.0,4887681,15524,9911,25435,5218,7
847,0,0,0,0,0,0,0,0,0,1,...,190194.5,270223.5,326193.5,105806.0,4887681,15524,9911,25435,5218,9
848,0,0,0,0,0,0,0,0,0,1,...,-44200.0,187836.0,184027.0,42437.0,4887681,15524,9911,25435,5218,9
849,0,0,0,0,0,0,0,0,0,1,...,-20832.5,46377.5,67942.0,-79734.5,4887681,15524,9911,25435,5218,9


In [179]:
df.head()

,Cluster__0,Cluster__1,Cluster__2,Cluster__3,Cluster__4,Cluster__5,Cluster__6,Cluster__7,Cluster__8,Cluster__9,...,onlineClassUsers,financeUsers,eCommerceUsers,sellingGoodsUsers,population,state_funds,fed_funds,total_spent,per_capita,cluster
0,0,1,0,0,0,0,0,0,0,0,...,-1.0,-1.0,-1.0,-1.0,4903185,15524,9911,25435,5218,1
1,0,0,0,0,0,0,0,1,0,0,...,-1.0,-1.0,-1.0,-1.0,4903185,15524,9911,25435,5218,7
2,0,1,0,0,0,0,0,0,0,0,...,-1.0,-1.0,-1.0,-1.0,4903185,15524,9911,25435,5218,1
3,0,0,0,0,1,0,0,0,0,0,...,-1.0,-1.0,-1.0,-1.0,4903185,15524,9911,25435,5218,4
4,0,0,0,0,0,1,0,0,0,0,...,-1.0,-1.0,-1.0,-1.0,4903185,15524,9911,25435,5218,5


In [181]:
df.to_csv('lawdf.csv',index=False)